# Super-Resolution e2e model from Pytorch -> Mobile

<p> In this tutorial, we describe how to use ONNX to convert a well-defined SuperResolution Pytorch model to some intermediate representation (IR) and use this IR to run the model in Caffe2 backend and also on Mobile. </p>
<br>
<p> This work is a joint collaboration among various awesome developers on Pytorch and Caffe2 </p>

In [ ]:
# Step 1: Let's import a bunch of stuff we are going to need for this
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import io
import numpy as np

from torch import nn
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo
import torch.onnx.export as export_model

import onnx
import onnx.backend.c2 as backend

<p>Now, let's create a SuperResolution model in Pytorch. This model definition can be found at https://github.com/pytorch/examples/blob/master/super_resolution/model.py</p>

In [ ]:
# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv4.weight)

# Step 2: Create the super-resolution model by using the above model definition.
py_model = SuperResolutionNet(upscale_factor=3)

<p><b>NOTE:</b> The above model definition has inplace=False since ONNX doesn't supprt inplace operations yet but will soon. Please watch changelog {INSERT CHANGELOG doc link here}</p>

<p> For the purpose of this tutorial, we will use a pre-trained super-resolution model. Note that this is optional.</p>

In [ ]:
# Step 3: define input, load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 2    # just a random number

# Input to the model
x = Variable(torch.randn(batch_size, 1, 224, 224), requires_grad=True)

# Initialize model with the pretrained weights
py_model.load_state_dict(model_zoo.load_url(model_url))

# set the train mode to false since we will only run the forward pass.
py_model.train(False)

<p>Now, we will export the model to a serialized ONNX IR proto which is readable/executable by other backends like Caffe2, CNTK etc. In the steps below, we'll see how to do the export, run the model in Caffe2 backend and verify the numerical correctness.</p>

In [ ]:
# Step 4: convert the model to ONNX, run in Caffe2 and verify numerical correctness

# first, create a file-like object where the serialized IR will be written
onnx_ir = io.BytesIO()

# Now, export the model to ONNX and run the model in PyTorch
torch_out = export_model(model, x, onnx_ir, export_params=True)

# Load the IR in ONNX to get the graph protobuf
graph = onnx.load(onnx_ir)

# prepare the caffe2 backend for executing the model
internal_rep = backend.prepare(graph)

# run the model in caffe2 backend
c2_out = backend.run(internal_rep, x)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

<b>NOTE:</b> <i>export_params=True</i> allows passing the model parameters with the serialized ONNX IR proto

<p>Using very simple steps above, we see how easy it is to convert a PyTorch model to an IR consumable by other backends and run the model there. Now, in next few steps, we'll see how to convert the model so that it can be run on mobile.</p>

### Running the model on mobile

<p>Caffe2 supports running networks on mobile very easily [information here: https://caffe2.ai/docs/mobile-integration.html]. 
We will use the Caffe2 mobile_exporter [https://github.com/caffe2/caffe2/blob/master/caffe2/python/predictor/mobile_exporter.py] to generate the two model protobufs that can run on mobile.</p>

In [ ]:
# extract the workspace and the graph proto from the internal representation
c2_workspace = internal_rep.workspace
c2_graph = internal_rep.predict_net

# Now import the caffe2 mobile exporter
from caffe2.python.predictor import mobile_exporter

# call the Export to get the predict_net, init_net
init_net, predict_net = mobile_exporter.Export(c2_workspace, c2_graph, c2_graph.input)

<p>Now, on your ios/Android device, you can use the above protobufs and use caffe2::Predictor (iOS) or Caffe2 instance (Android) for deploying them real-time. For more information, also checkout caffe2 https://caffe2.ai/docs/AI-Camera-demo-android.html</p>